# Chapter 20: Zero-Knowledge Proofs

---

Zero-knowledge proofs (ZKPs) are one of the most powerful and fascinating innovations in modern cryptography. They allow one party (the prover) to convince another party (the verifier) that a statement is true without revealing any information beyond the validity of the statement itself. In blockchain, ZKPs enable privacy, scalability, and novel use cases like private transactions and verifiable off-chain computation. This chapter introduces the core concepts, explores the major types of ZKPs, and provides hands-on examples using popular development tools.

---

## 20.1 Introduction to Zero-Knowledge

### 20.1.1 What is a Zero-Knowledge Proof?

Imagine you want to prove to a friend that you know the combination to a safe, but you don't want to reveal the combination. You could let them watch you open the safe, but that would show the combination. A zero-knowledge proof lets you convince them you know the secret without ever revealing it.

**Formal definition:** A zero-knowledge proof is a protocol between a prover and a verifier where the prover demonstrates knowledge of a secret (or that a statement is true) such that:
1. **Completeness**: If the statement is true, an honest prover can convince an honest verifier.
2. **Soundness**: If the statement is false, no cheating prover can convince the verifier (except with negligible probability).
3. **Zero-knowledge**: The verifier learns nothing except that the statement is true.

### 20.1.2 Properties: Completeness, Soundness, Zero-Knowledge

These three properties define any zero-knowledge protocol. Let's examine them in the context of a classic example: the **Ali Baba cave**.

```
The Ali Baba Cave Analogy:

                     ┌─────────────────┐
                     │                 │
                     │    Entrance     │
                     │                 │
                     └────────┬────────┘
                              │
              ┌───────────────┴───────────────┐
              │                               │
              │                               │
              ▼                               ▼
        ┌───────────┐                   ┌───────────┐
        │  Path A   │                   │  Path B   │
        └───────────┘                   └───────────┘
              │                               │
              └───────────────┬───────────────┘
                              │
                        ┌─────┴─────┐
                        │  Magic    │
                        │   Door    │
                        └───────────┘

• Peggy (prover) claims she knows the secret to open the magic door.
• Victor (verifier) stands at the entrance and can't see inside.
• Peggy enters the cave, picks a random path (A or B), but Victor doesn't see which.
• Victor then calls out a random path (A or B) for Peggy to exit from.
• If Peggy knows the secret, she can always exit from the requested path (by opening the door if needed).
• If she doesn't, she can only comply if she guessed the path correctly (probability 1/2).
• After many rounds, Victor becomes convinced Peggy knows the secret, but learns nothing about the secret itself.

This demonstrates:
- Completeness: If Peggy knows the secret, she always succeeds.
- Soundness: If Peggy doesn't know, she'll fail with probability 1/2 each round; after many rounds, probability of always succeeding becomes negligible.
- Zero-knowledge: Victor only sees Peggy exit from the requested path; no information about the secret is revealed.
```

### 20.1.3 Use Cases in Blockchain

Zero-knowledge proofs have transformative applications in blockchain:

| Use Case | Description | Example |
|----------|-------------|---------|
| **Privacy** | Hide transaction amounts, sender, or receiver | Zcash (shielded transactions), Tornado Cash |
| **Scalability** | Prove correctness of off-chain computation (ZK-rollups) | zkSync, StarkNet |
| **Identity** | Prove you have certain credentials without revealing them | Age verification, KYC compliance |
| **Verifiable Computation** | Prove that a computation was performed correctly | Off-chain execution, oracles |
| **Layer 2** | Validity proofs for rollups | zkEVM, Polygon zkEVM |

---

## 20.2 Types of ZK Proofs

Several families of zero-knowledge protocols exist, each with different trade-offs in proof size, verification time, and trust assumptions.

### 20.2.1 zk-SNARKs (Zero-Knowledge Succinct Non-Interactive Arguments of Knowledge)

zk-SNARKs are the most well-known type. Key properties:
- **Succinct**: Proofs are small (a few hundred bytes) and fast to verify (milliseconds).
- **Non-interactive**: A single message from prover to verifier (no back-and-forth).
- **Requires trusted setup**: A one-time ceremony generates parameters; if compromised, false proofs can be created.

**How they work (conceptually):**
1. The statement to prove is converted into an arithmetic circuit (a set of equations).
2. The prover generates a proof using the circuit and the secret witness.
3. The verifier checks the proof against a verification key.

**Example: Proving you know a hash preimage without revealing it.**

**Pros:**
- Very fast verification.
- Small proof size (ideal for on-chain verification).

**Cons:**
- Trusted setup (though newer schemes like Zcash's Sapling use multi-party computation to reduce risk).
- More complex to implement.

**Popular implementations:**
- libsnark (C++)
- snarkjs (JavaScript)
- Circom (domain-specific language)

### 20.2.2 zk-STARKs (Zero-Knowledge Scalable Transparent Arguments of Knowledge)

zk-STARKs were developed to address SNARKs' trusted setup requirement. They are:
- **Transparent**: No trusted setup (public randomness).
- **Scalable**: Prover time scales quasilinearly, verification logarithmically.
- **Post-quantum secure**: Based on hash functions, not elliptic curves.

**Trade-offs:**
- Larger proof size (tens to hundreds of KB).
- Higher verification cost (still fast, but more than SNARKs).

**How they work:**
- Use interactive oracle proofs (IOPs) and hash-based commitments.
- Prover commits to a polynomial, then responds to random challenges.

**Pros:**
- No trusted setup.
- Quantum-resistant.

**Cons:**
- Larger proofs (not ideal for on-chain where storage costs matter, but still usable).
- Less mature tooling.

**Popular implementations:**
- StarkWare's Cairo language and prover.
- Winterfell (Rust library).

### 20.2.3 Bulletproofs

Bulletproofs are another family, focusing on efficient range proofs (proving a value is within a range without revealing it). They are:
- **No trusted setup**.
- **Short proofs** (logarithmic in the range size).
- **Slow to verify** compared to SNARKs.

**Use cases:**
- Confidential transactions (prove amounts are non-negative without revealing).
- Private smart contracts.

**Pros:**
- Trustless setup.
- Relatively small proofs.

**Cons:**
- Slower verification (not ideal for on-chain).

**Popular implementations:**
- dalek-cryptography/bulletproofs (Rust).
- Monero uses Bulletproofs for transaction amounts.

### 20.2.4 Comparison and Trade-offs

| Property | zk-SNARKs | zk-STARKs | Bulletproofs |
|----------|-----------|-----------|--------------|
| Proof Size | ~200 bytes | ~45-200 KB | ~1-5 KB |
| Verification Time | ~10 ms | ~10-100 ms | ~1-10 ms |
| Prover Time | Fast | Slower | Moderate |
| Trusted Setup | Yes | No | No |
| Quantum-Resistant | No | Yes | No |
| Use Cases | ZK-rollups, private payments | Scalability, future-proof | Range proofs, Monero |

---

## 20.3 ZK Development Tools

Building ZK applications requires specialized languages and frameworks. Here are the most popular ones.

### 20.3.1 Circom Language

Circom is a domain-specific language for writing arithmetic circuits. It compiles to R1CS (Rank-1 Constraint System), which can then be used with snarkjs to generate proofs.

**Installation:**
```bash
npm install -g circom
npm install -g snarkjs
```

**Example: A circuit that proves knowledge of a preimage of a hash (simplified).**
```c
// circuit.circom
pragma circom 2.0.0;

template Preimage() {
    signal input preimage;      // private input
    signal output hash;         // public output

    // Assume a simple hash function: hash = preimage * preimage (for demo only!)
    // In practice, use a real hash like Poseidon or SHA256.
    signal square;
    square <== preimage * preimage;
    hash <== square;
}

component main = Preimage();
```

**Compile and generate proof:**
```bash
circom circuit.circom --r1cs --wasm --sym
snarkjs powersoftau new bn128 12 pot12_0000.ptau -v
snarkjs powersoftau contribute pot12_0000.ptau pot12_0001.ptau --name="First contribution" -v
snarkjs powersoftau prepare phase2 pot12_0001.ptau pot12_final.ptau -v
snarkjs groth16 setup circuit.r1cs pot12_final.ptau circuit_0000.zkey
snarkjs zkey contribute circuit_0000.zkey circuit_final.zkey --name="1st Contributor" -v
snarkjs zkey export verificationkey circuit_final.zkey verification_key.json

# Generate witness and proof
node generate_witness.js circuit.wasm input.json witness.wtns
snarkjs groth16 prove circuit_final.zkey witness.wtns proof.json public.json

# Verify
snarkjs groth16 verify verification_key.json public.json proof.json
```

### 20.3.2 ZoKrates

ZoKrates is a toolbox for zk-SNARKs on Ethereum. It provides a high-level language, a compiler, and tools to generate and verify proofs on-chain.

**Installation:**
```bash
curl -LSfs get.zokrat.es | sh
```

**Example: Prove you know a factor of a number.**
```python
// main.zok
def main(private field a, private field b, field c) -> bool {
    return a * b == c;
}
```

**Compile and run:**
```bash
zokrates compile -i main.zok
zokrates setup
zokrates compute-witness -a 3 4 12
zokrates generate-proof
zokrates export-verifier
```

This generates a Solidity verifier contract that can be deployed to verify proofs on-chain.

### 20.3.3 Noir (Aztec)

Noir is a domain-specific language for ZK proofs developed by Aztec. It's designed to be more developer-friendly and compiles to multiple backends (Barretenberg, Arkworks).

**Example:**
```rust
// main.nr
fn main(x : Field, y : pub Field) {
    constrain x * x == y;
}
```

**Compile and prove:**
```bash
nargo compile
nargo prove
nargo verify
```

Noir is gaining popularity due to its syntax similarity to Rust and its integration with Aztec's privacy-focused rollup.

---

## 20.4 Building ZK Applications

Let's explore two practical applications of zero-knowledge proofs: privacy-preserving transactions and identity verification.

### 20.4.1 Privacy-Preserving Transactions

Zcash pioneered private transactions using zk-SNARKs. The idea: hide the sender, receiver, and amount while still allowing consensus on the total supply.

**Simplified concept:**
- A shielded transaction proves that the total input notes (old coins) equal the total output notes (new coins) plus a fee.
- Each note is a commitment (hash) of a value and a secret key.
- The proof shows that all notes are valid without revealing which notes are being spent.

**Example using Circom/snarkjs:**
We won't build a full Zcash, but we can sketch a simple private transfer circuit.

```c
// PrivateTransfer.circom
pragma circom 2.0.0;

include "circomlib/circuits/poseidon.circom";

template PrivateTransfer() {
    signal input root;          // Merkle root of note tree (public)
    signal input nullifier;      // nullifier to prevent double-spend (public)
    signal input fee;            // fee (public)

    signal input notePreimage;   // private: value + secret
    signal input pathIndices;    // private: path to leaf
    signal input siblings[10];   // private: Merkle siblings

    component hash = Poseidon(2);
    hash.inputs[0] <== notePreimage;  // In real life, split into value and secret
    hash.inputs[1] <== 0;
    signal commitment <== hash.out;

    // Verify commitment is in the tree with given root
    component merkle = MerkleTreeChecker(10);
    merkle.leaf <== commitment;
    merkle.pathIndices <== pathIndices;
    for (var i=0; i<10; i++) {
        merkle.siblings[i] <== siblings[i];
    }
    merkle.root === root;

    // Compute nullifier (should be unique per spent note)
    component nullifierHash = Poseidon(2);
    nullifierHash.inputs[0] <== notePreimage;
    nullifierHash.inputs[1] <== 1;
    nullifierHash.out === nullifier;

    // Fee logic (ensure value > fee) omitted for brevity
}

component main {public [root, nullifier, fee]} = PrivateTransfer();
```

This circuit checks that:
1. The note (commitment) exists in a Merkle tree with a known root.
2. The nullifier is correctly derived (prevents double-spending).
3. The fee is correct (not shown).

The proof is verified on-chain; the contract updates the nullifier set and root.

### 20.4.2 Identity Verification

Another powerful use case: prove you meet certain criteria without revealing personal data. For example, prove you are over 18 without revealing your birthdate.

**Using ZoKrates:**
```
// age.zok
def main(private u32 age) -> bool {
    return age >= 18;
}
```

The user inputs their age privately, the circuit outputs true if age >= 18. The proof can be verified by a smart contract that doesn't learn the age.

**On-chain verifier (Solidity snippet):**
```solidity
// Generated by ZoKrates
contract Verifier {
    function verifyTx(
        uint[2] memory a,
        uint[2][2] memory b,
        uint[2] memory c,
        uint[1] memory input
    ) public view returns (bool) {
        // input[0] is the public output (1 if true)
        // ...
    }
}
```

### 20.4.3 Code Examples

Let's walk through a complete example using Circom and snarkjs for a simple "I know the preimage of a hash" proof, suitable for on-chain verification.

**Step 1: Circuit (hash.circom)**
```c
pragma circom 2.0.0;

include "../node_modules/circomlib/circuits/sha256/sha256.circom";

template Preimage() {
    signal input preimage;      // private (would be 256-bit in practice)
    signal output hash;         // public

    // In reality, we'd split preimage into bytes, use SHA256
    // For simplicity, assume a trivial "hash" (preimage * preimage)
    // But let's use a proper hash: we'll use Poseidon from circomlib
    component poseidon = Poseidon(1);
    poseidon.inputs[0] <== preimage;
    hash <== poseidon.out;
}

component main {public [hash]} = Preimage();
```

**Step 2: Input (input.json)**
```json
{"preimage": "123"}
```

**Step 3: Compile and generate proof (script)**
```bash
circom hash.circom --r1cs --wasm --sym
snarkjs groth16 setup hash.r1cs pot12_final.ptau hash_0000.zkey
snarkjs zkey contribute hash_0000.zkey hash_final.zkey --name="Contributor" -v
snarkjs zkey export verificationkey hash_final.zkey verification_key.json

# Generate witness
cd hash_js
node generate_witness.js hash.wasm ../input.json witness.wtns
cd ..

# Prove
snarkjs groth16 prove hash_final.zkey hash_js/witness.wtns proof.json public.json
```

**Step 4: Export Solidity verifier**
```bash
snarkjs zkey export solidityverifier hash_final.zkey verifier.sol
```

**Step 5: Verify on-chain**
Deploy `verifier.sol` and call `verifyTx` with the proof and public inputs.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 20 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Zero-knowledge proofs enable one party to prove a statement    │
│  true without revealing any additional information.             │
│                                                                 │
│  Three properties: Completeness, Soundness, Zero-Knowledge.     │
│                                                                 │
│  Major types:                                                  │
│    • zk-SNARKs: Small proofs, fast verification, trusted setup │
│    • zk-STARKs: No trusted setup, larger proofs, quantum-safe  │
│    • Bulletproofs: No trusted setup, efficient range proofs    │
│                                                                 │
│  Development tools:                                            │
│    • Circom: Domain-specific language for circuits             │
│    • ZoKrates: High-level language with Solidity export        │
│    • Noir: Rust-like language by Aztec                         │
│                                                                 │
│  Applications:                                                 │
│    • Private transactions (Zcash)                              │
│    • Identity verification (age proofs)                        │
│    • Scalability (ZK-rollups)                                  │
│                                                                 │
│  ZKPs are revolutionizing blockchain by adding privacy and     │
│  scalability. They are complex but essential for the next      │
│  generation of decentralized applications.                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 21 – Smart Contract Upgradeability. We'll explore why upgradeability is needed, the risks involved, and patterns like proxy contracts (Transparent, UUPS, Beacon) to safely upgrade smart contracts.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. token_standards.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='21. smart_contract_upgradeability.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
